In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import json
import pickle

In [3]:
df = pd.read_csv('/content/drive/MyDrive/집교 2_Team P/user-track-listen_count.csv')

In [4]:
from sklearn.preprocessing import LabelEncoder
user_encoder = LabelEncoder()
track_encoder = LabelEncoder()
lyrics_encoder = LabelEncoder()
df['user_id'] = user_encoder.fit_transform(df['user_id'])
df['track_id'] = track_encoder.fit_transform(df['track_id'])

In [5]:
# !pip install torch torchvision -U

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import pandas as pd
import numpy as np
from tqdm import tqdm

# 데이터 불러오기 (예시: CSV 파일)
# Label Encoding
df.loc[df['listen_count_bin'] == '10~2704', 'listen_count_bin'] = 10
df['listen_count_bin'] = df['listen_count_bin'].astype(int)
# PyTorch DataLoader에 맞게 데이터 변환
def df_to_tensor(dataset):
    users = torch.tensor(dataset['user_id'].values, dtype=torch.int)
    items = torch.tensor(dataset['track_id'].values, dtype=torch.int)
    ratings = torch.tensor(dataset['listen_count_bin'].values, dtype=torch.float)

    return users, items, ratings

train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

train_users, train_items, train_ratings = df_to_tensor(train_df)
test_users, test_items, test_ratings = df_to_tensor(test_df)

train_data = TensorDataset(train_users, train_items, train_ratings)
test_data = TensorDataset(test_users, test_items, test_ratings)

train_loader = DataLoader(train_data, batch_size=256, shuffle=True)
test_loader = DataLoader(test_data, batch_size=256, shuffle=False)
num_users = (df['user_id'].nunique())
num_items = (df['track_id'].nunique())
print(num_users)
print(num_items)

24458
31110


In [6]:
# NCF 모델 정의
class NCF(nn.Module):
    def __init__(self, num_users, num_items, embedding_size):
        super(NCF, self).__init__()
        self.user_embedding = nn.Embedding(num_users, embedding_size)
        self.item_embedding = nn.Embedding(num_items, embedding_size)
        self.fc_layers = nn.Sequential(
            nn.Linear(embedding_size * 2, embedding_size),
            nn.ReLU(),
            nn.Linear(embedding_size, int(embedding_size/2)),
            nn.ReLU(),
            nn.Linear(int(embedding_size/2), int(embedding_size/4)),
            nn.ReLU(),
            nn.Linear(int(embedding_size/4), int(embedding_size/8)),
            nn.ReLU(),
            nn.Linear(int(embedding_size/8), 1)
        )

    def forward(self, user, item):
        user_embedding = self.user_embedding(user)
        item_embedding = self.item_embedding(item)
        x = torch.cat((user_embedding, item_embedding), dim=1)
        x = self.fc_layers(x)
        return x

# CUDA 디바이스 설정
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)
def train_(embedding,n_epoch):
  # NCF 모델 정의 및 GPU로 이동
  model = NCF(num_users=num_users, num_items=num_items, embedding_size=embedding)
  model.to(device)
  criterion = nn.MSELoss()
  optimizer = optim.Adam(model.parameters(), lr=0.001)
  # tqdm을 사용하여 학습 및 테스트 진행 상황 확인
  num_epochs = n_epoch
  for epoch in range(num_epochs):
      model.train()
      total_loss = 0
      for user, item, rating in tqdm(train_loader, desc=f'Epoch {epoch+1}/{num_epochs}'):
          optimizer.zero_grad()
          user, item, rating = user.to(device), item.to(device), rating.to(device)  # GPU로 이동
          output = model(user, item)
          loss = criterion(output, rating.unsqueeze(1))
          loss.backward()
          optimizer.step()
          total_loss += loss.item()

      avg_loss = total_loss / len(train_loader)
      print(f'Epoch {epoch+1}/{num_epochs}, Avg. Loss: {avg_loss:.4f}')

      # 각 에폭이 끝날 때마다 테스트 데이터에 대한 예측 수행
      model.eval()
      all_predictions = []
      with torch.no_grad():
          for user, item, _ in tqdm(test_loader, desc=f'Testing Epoch {epoch+1}'):
              user, item = user.to(device), item.to(device)  # GPU로 이동
              output = model(user, item)
              all_predictions.append(output)

      # RMSE 계산
      predictions = torch.cat(all_predictions).squeeze().cpu().numpy()  # CPU로 이동 후 numpy로 변환
      rmse = np.sqrt(mean_squared_error(test_df['listen_count_bin'].values, predictions))
      print(f'Epoch {epoch+1}/{num_epochs}, RMSE on test set: {rmse}')


Using device: cuda


In [ ]:
train_(64,10)

Epoch 1/10: 100%|██████████| 14499/14499 [01:26<00:00, 167.37it/s]


Epoch 1/10, Avg. Loss: 6.1301


Testing Epoch 1: 100%|██████████| 3625/3625 [00:11<00:00, 312.20it/s]


Epoch 1/10, RMSE on test set: 2.3986429441561685


Epoch 2/10: 100%|██████████| 14499/14499 [01:26<00:00, 167.95it/s]


...

Epoch 9/10, RMSE on test set: 2.3677210345863657


Epoch 10/10: 100%|██████████| 14499/14499 [01:25<00:00, 169.01it/s]


Epoch 10/10, Avg. Loss: 4.5775


Testing Epoch 10: 100%|██████████| 3625/3625 [00:11<00:00, 313.77it/s]


Epoch 10/10, RMSE on test set: 2.375622445097809


In [ ]:
train_(128,10)

Epoch 1/10: 100%|██████████| 14499/14499 [01:39<00:00, 145.65it/s]


Epoch 1/10, Avg. Loss: 6.0735


Testing Epoch 1: 100%|██████████| 3625/3625 [00:11<00:00, 309.33it/s]


Epoch 1/10, RMSE on test set: 2.3943491028875346


Epoch 2/10: 100%|██████████| 14499/14499 [01:41<00:00, 142.38it/s]


...

Epoch 9/10, RMSE on test set: 2.3793792553928723


Epoch 10/10: 100%|██████████| 14499/14499 [01:42<00:00, 141.83it/s]


Epoch 10/10, Avg. Loss: 3.6408


Testing Epoch 10: 100%|██████████| 3625/3625 [00:11<00:00, 307.09it/s]

Epoch 10/10, RMSE on test set: 2.3969830468909454


In [ ]:
train_(256,10  )

Epoch 1/10: 100%|██████████| 14499/14499 [02:12<00:00, 109.53it/s]


Epoch 1/10, Avg. Loss: 6.0026


Testing Epoch 1: 100%|██████████| 3625/3625 [00:11<00:00, 302.82it/s]


Epoch 1/10, RMSE on test set: 2.3903326763870485


Epoch 2/10: 100%|██████████| 14499/14499 [02:14<00:00, 108.02it/s]


...

Epoch 9/10, RMSE on test set: 2.392561669125588


Epoch 10/10: 100%|██████████| 14499/14499 [02:13<00:00, 108.63it/s]


Epoch 10/10, Avg. Loss: 2.8348


Testing Epoch 10: 100%|██████████| 3625/3625 [00:11<00:00, 310.60it/s]

Epoch 10/10, RMSE on test set: 2.4471423189679293


In [7]:
train_(512,10)

Epoch 1/10: 100%|██████████| 14544/14544 [01:37<00:00, 149.55it/s]


Epoch 1/10, Avg. Loss: 5.9653


Testing Epoch 1: 100%|██████████| 3636/3636 [00:12<00:00, 291.76it/s]


Epoch 1/10, RMSE on test set: 2.3938648728518004


Epoch 2/10: 100%|██████████| 14544/14544 [01:35<00:00, 151.60it/s]


...

Epoch 9/10, RMSE on test set: 2.4650860014491527


Epoch 10/10: 100%|██████████| 14544/14544 [01:37<00:00, 149.32it/s]


Epoch 10/10, Avg. Loss: 2.2722


Testing Epoch 10: 100%|██████████| 3636/3636 [00:12<00:00, 291.39it/s]

Epoch 10/10, RMSE on test set: 2.4885313362535455
